In [257]:
q = 2 
m = 4 
p = x^4 + x + 1
n = q^m - 1
t = 3 
F.<a> = GF(q^m, modulus=p) 
R.<x> = PolynomialRing(F) 

In [258]:
# received word 
r = [0, 0, 0, a^4, 0, 0, a^7, 0, a^2, 1, a^10, a, a^12, a^4, a^13]
assert len(r) == n

In [259]:
print(q^m -1)
B_i = [a^i for i in range(q^m - 1)]
print(B_i)
assert len(B_i) == n

15
[1, a, a^2, a^3, a + 1, a^2 + a, a^3 + a^2, a^3 + a + 1, a^2 + 1, a^3 + a, a^2 + a + 1, a^3 + a^2 + a, a^3 + a^2 + a + 1, a^3 + a^2 + 1, a^3 + 1]


E(x) = (x - e1)(x - e2)(x - e3) - this because t = 3 <br/>
Q(x) = q0 + q1x + ... + q11^11 <br/>
Q(B_i) = riE(B_i), i = 0, 1, ...14    

In [260]:
# create matrix to solve 

# two dim. list 

print(f"n-t {n-t}, n = {n}, t = {t}")
print("B_i : ", len(B_i))

rows = [ 
    # Q is monic, thus first elem. is 1 
    
    # add B_i^i for 1 -> 12
    [B_i[bi]^i for i in range(n-t)] 
    
    + # add error vec. 
      [r[bi]] 
    + [r[bi] * B_i[bi]]
    + [r[bi] * B_i[bi]^2]
    # [(r[bi]*B_i[bi])^i for i in range(t)]
    
    for bi in range(len(B_i))]

matrix_ = Matrix(F, rows) 
print(matrix_.dimensions())
print(matrix_.determinant())


n-t 12, n = 15, t = 3
B_i :  15
(15, 15)
a


In [261]:
for _ in B_i: 
    print(_)

1
a
a^2
a^3
a + 1
a^2 + a
a^3 + a^2
a^3 + a + 1
a^2 + 1
a^3 + a
a^2 + a + 1
a^3 + a^2 + a
a^3 + a^2 + a + 1
a^3 + a^2 + 1
a^3 + 1


In [262]:
for _ in r: 
    print(_)

0
0
0
a + 1
0
0
a^3 + a + 1
0
a^2
1
a^2 + a + 1
a
a^3 + a^2 + a + 1
a + 1
a^3 + a^2 + 1


In [263]:
# add vector of known values 
v = vector(F, [r[i]*B_i[i]^3 for i in range(15)])

for _ in v: 
    print(_)

0
0
0
a^3 + a^2 + 1
0
0
a^2 + a + 1
0
a^3 + a^2 + a
a^3 + a^2 + a + 1
a^2 + a + 1
a + 1
a^3
a^3 + a^2 + 1
a^2 + a + 1


In [264]:
# M = matrix_.hstack(v) 
print(type(matrix_))

M = matrix_.solve_right(v) #  matrix_.augment(v.column())
# print(M.dimensions())
solved_matrix = list(M) # list(matrix(solved)) 

for _ in solved_matrix: 
    print(_)

<class 'sage.matrix.matrix_gf2e_dense.Matrix_gf2e_dense'>
0
a^3 + a + 1
a + 1
a^3 + 1
0
a^3 + a + 1
a
a^2 + 1
a^2 + a + 1
a^2
a^3 + a
a^2
a^3 + a^2 + a
a^3 + a^2
a^3 + a


1

In [266]:
output = {}
c = 0 

Q_x = 0
E_x = 0 

for qi in range(n-t):
    output[f"q{c}"] = solved_matrix[c]
    Q_x += solved_matrix[c] * x^qi
    c += 1

for ei in range(3): 
    output[f"e{c % 12}"] = solved_matrix[c]
    E_x += solved_matrix[c] * x^ei
    c += 1
# Remember to add x^3 to E_x 
E_x += x^3
output

{'q0': 0,
 'q1': a^3 + a + 1,
 'q2': a + 1,
 'q3': a^3 + 1,
 'q4': 0,
 'q5': a^3 + a + 1,
 'q6': a,
 'q7': a^2 + 1,
 'q8': a^2 + a + 1,
 'q9': a^2,
 'q10': a^3 + a,
 'q11': a^2,
 'e0': a^3 + a^2 + a,
 'e1': a^3 + a^2,
 'e2': a^3 + a}

In [267]:
Q_x

a^2*x^11 + (a^3 + a)*x^10 + a^2*x^9 + (a^2 + a + 1)*x^8 + (a^2 + 1)*x^7 + a*x^6 + (a^3 + a + 1)*x^5 + (a^3 + 1)*x^3 + (a + 1)*x^2 + (a^3 + a + 1)*x

In [268]:
E_x

x^3 + (a^3 + a)*x^2 + (a^3 + a^2)*x + a^3 + a^2 + a

In [269]:
# compute the quotient F_x of Q_x divided by E_x
F_q, rem = R(Q_x).quo_rem(R(E_x))

In [270]:
f = Q_x/E_x
f

(a^2*x^10 + (a^2 + a + 1)*x^9 + a^3*x^8 + (a^3 + a^2 + a)*x^7 + (a^3 + a^2 + a)*x^6 + (a^3 + 1)*x^5 + (a^3 + a^2)*x^4 + a^2*x^3 + a^2*x^2 + (a^3 + a^2 + a)*x)/(x^2 + a^2*x + 1)

In [271]:

#out = [F_q(a^i) for i in range(n)]
out = [f(a^i) for i in range(n)]

In [272]:
out

[0,
 0,
 0,
 a + 1,
 0,
 0,
 a^3 + a + 1,
 0,
 a^2,
 1,
 a^2 + a + 1,
 a^2 + 1,
 a^3 + a^2 + a + 1,
 a + 1,
 a^3 + a^2 + 1]

In [273]:
topower = {a^i: f'a^{i}' for i in range(15)}
topower[0] = '0'
topower[a^0] = 1

In [287]:
org = ' '.join(list(map(str, [x. for x in r])))
corrected = ' '.join(list(map(str, [topower.get (i) for i in out])))

print(f"org: \t{r}")
print(f"cor_: \t{corrected}")

org: 	[0, 0, 0, a + 1, 0, 0, a^3 + a + 1, 0, a^2, 1, a^2 + a + 1, a, a^3 + a^2 + a + 1, a + 1, a^3 + a^2 + 1]
cor_: 	0 0 0 a^4 0 0 a^7 0 a^2 1 a^10 a^8 a^12 a^4 a^13
